In [ ]:
#| default_exp testbed/web_server

In [ ]:
#| export

from __future__ import annotations


In [ ]:
#| hide
# %reload_ext autoreload
# %autoreload 0


# install (Colab)

In [ ]:
MOUNT_DRIVE = True
DEV_INSTALL = True
GDRIVE_MOUNT_POINT = 'drive'


In [ ]:
import os
from pathlib import Path
import fastcore.all as FC
from rich import print as cprint
from rich.text import Text

def info(msg: str):
    text = Text(msg)
    text.stylize("bold red", 0, 6)
    cprint("_" * 10, text, "_" * 10)


if FC.IN_COLAB:
    if MOUNT_DRIVE:
        mnt_point = f"/content/{GDRIVE_MOUNT_POINT}"
        if not Path(mnt_point).exists():
            info("Mounting Google Drive")
            from google.colab import drive

            drive.mount(mnt_point, force_remount=True)


Colab has issues with PanelClenar PIL version.: uninstall Colab one, restart wen prompted and rerun from the top.


In [ ]:
if FC.IN_COLAB:
    from packaging import version
    import PIL
    pil_version = version.parse(PIL.__version__)
    if pil_version < version.parse("10"):
        info('Uninstalling Pillow')
        !pip uninstall Pillow
        info('Installing Pillow')
        !pip install Pillow


In [ ]:
if FC.IN_COLAB:
    info('Installing PanelCleaner')
    if DEV_INSTALL:
        assert MOUNT_DRIVE, "DEV_INSTALL need a mounted google drive drive"
        info('Installing PanelCleaner from Google Drive')
        os.chdir('/content/drive/MyDrive/Shared/PanelCleaner/')
        !pip install -e .
    else:
        info('Installing PanelCleaner from Github')
        !pip install -q git+https://github.com/civvic/PanelCleaner.git@testbed-colab


In [ ]:
if FC.IN_COLAB:
    info('Installing PanelCleaner Colab requirements')
    import importlib.resources
    if DEV_INSTALL:
        os.chdir('pcleaner/_testbed')
    
    try:
        package_path = importlib.resources.files('pcleaner')
        info('Installing PanelCleaner testbed requirements')
        p = (Path(package_path)/'_testbed/requirements-colab.txt')
        if p.exists():
            !pip install -r {p}
        else:
            print(f"colab requirements {p} not found")
    except Exception:
        info("Couldn't install PanelCleaner Colab requirements")


# Basic web server for serving images from Google Drive


# Prologue

In [ ]:
#| export
import getpass
import http.server
import os
import signal
import socketserver
import threading
import uuid
from http import HTTPStatus
from pathlib import Path
from typing import Protocol

import portpicker
import psutil
import requests
import rich
from IPython.display import display
from IPython.display import HTML
from loguru import logger
from pyngrok import conf
from pyngrok import ngrok
from rich.console import Console


In [ ]:
#| exporti
from pcleaner._testbed.testbed.bottle import Bottle
from pcleaner._testbed.testbed.bottle import HTTPError
from pcleaner._testbed.testbed.bottle import response
from pcleaner._testbed.testbed.bottle import run
from pcleaner._testbed.testbed.bottle import static_file


In [ ]:
import fastcore.all as FC
import fastcore.xtras  # patch Path with some utils
from fastcore.test import *  # type: ignore

import  pcleaner._testbed.testbed.bottle as bottle


# Helpers

In [ ]:
# pretty print by default
# %load_ext rich

In [ ]:
#| exporti
console = Console(width=104, tab_size=4, force_jupyter=True)
cprint = console.print


In [ ]:
#| export

def display_ngrok_warning(url):
    did = 'ngrokFrame' + str(uuid.uuid4())
    html_code = f"""
<div style="font-size: 13pt;" id="{did}">
  <div style="background-color: aliceblue;">
    <p><b>Ngrok</b> displays a <b>warning page</b> as a security measure to prevent unintentional access to your local servers. This page requires you to <b>confirm</b> that you wish to proceed to the content.</p>
    <p style="font-weight: bold;">Please review the <em>ngrok</em> warning page displayed below. If prompted, click '<b>Visit Page</b>' to proceed.</p>
    Don't worry if you see a <b>404</b> or <b>403</b> error. Then, you can click the '<b>Close</b>' button below to hide this section.</p>
  </div>
    <iframe src="{url}" width="100%" height="600px" style="border:none;"></iframe>
    <button onclick="document.getElementById('{did}').innerHTML='';">Close</button>
</div>
"""

    display(HTML(html_code))


In [ ]:
#| export

class WebServer(Protocol):
    @property
    def public_url(self) -> str | None: ...
    @property
    def unc_share(self) -> Path | None: ...
    @property
    def prefix(self) -> str: ...
    @property
    def running(self) -> bool: ...
    def __init__(self, directory: Path | str = ""): ...
    def start(self): ...
    def stop(self): ...


def setup_ngrok(server_cls: type[WebServer], images_dir: str | Path):
    cprint(
        "Enter your ngrok authtoken, which can be copied from "
        "https://dashboard.ngrok.com/get-started/your-authtoken"
    )
    auth_token = getpass.getpass()
    conf.get_default().auth_token = auth_token
    ngrok.set_auth_token(auth_token)

    server = server_cls(directory=str(images_dir))
    try:
        server.start()
    except Exception as e:
        cprint(f"Error starting server: {e}")
        return None

    display_ngrok_warning(f"{server.public_url}/{server.prefix}/pcleaner.png")
    return server


----

Modify `cache_dir` path to point to the directory containing the images you want to serve


In [ ]:
cache_dir = Path('../experiment/cache')
test_eq(cache_dir.exists(), True)


# WebServerStdlib
> simple web server based on `http.server` and `ngrok` as reverse proxy


In [ ]:
#| exporti

class ImageHTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
    def end_headers(self):
        for k,v in {
                'ngrok-skip-browser-warning': 'true',
                'User-Agent': 'MyCustomUserAgent/1.0',
                'Cache-Control': 'public, max-age=86400'
            }.items():
            self.send_header(k, v)
        super().end_headers()

    def do_GET(self):
        if self.is_image_request(self.path):
            super().do_GET()
        else:
            self.send_error(HTTPStatus.FORBIDDEN, "Only image files are accessible.")

    def is_image_request(self, path):
        allowed_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.webp')
        _, ext = os.path.splitext(path)
        return ext.lower() in allowed_extensions
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, directory=self.directory, **kwargs)


In [ ]:
#| export

class WebServerStdlib:
    """
    A simple web server for serving images from a local directory using http.server and ngrok.
    
    It is intended to be used in environments like Google Colab, where direct
    web server hosting might not be feasible. It uses ngrok to allow images to be accessed 
    via a public URL.
    
    Attributes:
        directory (str): The directory from which files are served.
        port (int): The local port on which the server listens.
        public_url (str): The ngrok public URL where the server is accessible.
        tunnel (ngrok.NgrokTunnel): The ngrok tunnel object.
    
    Methods:
        start(): Starts the web server and the ngrok tunnel.
        stop(): Stops the web server and disconnects the ngrok tunnel.
        make_request(path="/"): Makes a request to the ngrok URL to fetch data from the server.
    """
    
    def __init__(self, directory: Path | str=""):
        port = portpicker.pick_unused_port()
        self.port = port
        if isinstance(directory, str): 
            directory = Path(directory)
        assert directory.exists(), f"Directory {directory} does not exist"
        self.directory = str(directory.resolve())
        self.thread = None
        self.httpd = None
        self.public_url = None
        self.prefix = ''
        self.unc_share = None
        self.tunnel = None

    @property
    def running(self):
        return self.thread is not None and self.thread.is_alive()
    
    def start_server(self):
        Handler = ImageHTTPRequestHandler 
        Handler.directory = self.directory
        try:
            with socketserver.TCPServer(("", self.port), Handler) as httpd:
                self.httpd = httpd
                httpd.serve_forever()
        except OSError as e:
            cprint(f"Error: {e}")

    def start(self):
        if self.thread is None or not self.thread.is_alive():
            self.thread = threading.Thread(target=self.start_server)
            self.thread.start()
            self.tunnel = ngrok.connect(self.port)  # type: ignore
            self.public_url = self.tunnel.public_url
            if self.public_url is not None:
                self.unc_share = Path(self.public_url.replace('https:', ''))
            cprint(f"ngrok tunnel: {self.tunnel}")
            cprint(f"Public URL: {self.public_url}")
        else:
            cprint("Server is already running")

    def stop(self):
        if self.httpd:
            self.httpd.shutdown()
            self.httpd.server_close()
        if self.tunnel and self.tunnel.public_url:
            ngrok.disconnect(self.tunnel.public_url)  # Use the stored tunnel object's URL
            cprint("Ngrok tunnel disconnected")
            ngrok.kill()
        if self.thread:
            self.thread.join()
        self.thread = self.public_url = self.unc_share = None
        cprint("Server stopped")

    def make_request(self, path="/"):
        """Makes a request to the ngrok URL with headers to bypass the ngrok warning."""
        if self.public_url:
            url = f"{self.public_url}{path}"
            headers = {
                "ngrok-skip-browser-warning": "true",
                "User-Agent": "MyCustomUserAgent/1.0"
            }
            response = requests.get(url, headers=headers)
            return response.text
        else:
            return "Server not started or public URL not available."

    def __enter__(self):
        self.start()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.stop()


In [ ]:
serve_dir = cache_dir
test_eq(serve_dir.exists(), True)
serve_dir

Path('../experiment/cache')

In [ ]:
server = setup_ngrok(WebServerStdlib, serve_dir)


Enter your ngrok authtoken, which can be copied from 
https://dashboard.ngrok.com/get-started/your-authtoken

ngrok tunnel: NgrokTunnel: "https://36a0-83-33-227-209.ngrok-free.app" -> "http://localhost:55435"

Public URL: https://36a0-83-33-227-209.ngrok-free.app

127.0.0.1 - - [21/May/2024 13:22:26] "GET //pcleaner.png HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2024 13:22:46] "GET /Strange_Tales_172005/.crop/Strange_Tales_172005_0_Default.png HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2024 13:22:48] "GET /Strange_Tales_172005/.crop/Strange_Tales_172005_1_Default.png HTTP/1.1" 200 -
t=2024-05-21T13:22:51+0200 lvl=warn msg="Stopping forwarder" name=http-55435-1681ca10-634d-4a9f-b4c3-e9ae3c2b48e2 acceptErr="failed to accept connection: Listener closed"


In [ ]:
assert server is not None and server.public_url is not None
public_url: str = server.public_url


In [ ]:
img_path = 'Strange_Tales_172005/.crop/Strange_Tales_172005_0_Default.png'


In [ ]:
f'<img src="{public_url}/{img_path}"/>'

'<img src="https://36a0-83-33-227-209.ngrok-free.app/Strange_Tales_172005/.crop/Strange_Tales_172005_0_Default.png"/>'

In [ ]:
display(HTML(f'<img src="{public_url}/{img_path}"/>'))

In [ ]:
display(HTML(f'<img src="{public_url}/Strange_Tales_172005/.crop/Strange_Tales_172005_1_Default.png"/>'))


In [ ]:
if server is not None: 
    server.stop()


Ngrok tunnel disconnected

Server stopped

In [ ]:
test_eq(ngrok.get_tunnels(), [])


In [ ]:
PORT = server.port
PORT


52794

In [ ]:
_PID = !lsof -ti :$PORT  # Find the process using PORT  # type: ignore
if len(_PID) > 0: _PID = _PID[0]
_PID


[]

In [ ]:
# !kill -9 $_PID

# WebServerBottle
> simple web server based on `Bottle` and `ngrok` as reverse proxy


In [ ]:
#| exporti

app = Bottle()

@app.route('/images/<filename:path>')  # type: ignore
def serve_image(filename):
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.webp')):
        return HTTPError(404, "File not found")
    response.set_header('Cache-Control', 'public, max-age=86400')  # Set caching headers
    return static_file(filename, root=app.config['image_dir'])

@app.route('/shutdown')  # type: ignore
def shutdown():
    current_process = psutil.Process()
    current_process.send_signal(signal.SIGTERM)


In [ ]:
#| export

class WebServerBottle:
    """
    A simple web server for serving images from a local directory using ngrok.
    This class uses the Bottle framework to handle HTTP requests and ngrok to expose the 
    server to the internet.It is designed to be used in environments like Google Colab, 
    where direct web server hosting might not be feasible.
    
    Attributes:
        directory (Path | str): The directory from which files are served.
        port (int): The local port on which the server listens.
        public_url (str): The ngrok public URL where the server is accessible.
        tunnel (ngrok.NgrokTunnel): The ngrok tunnel object.
    
    Methods:
        start(): Starts the web server and the ngrok tunnel.
        stop(): Stops the web server and disconnects the ngrok tunnel.
    """
    
    def __init__(self, directory: Path | str = ""):
        self.port = portpicker.pick_unused_port()
        if isinstance(directory, str):
            directory = Path(directory)
        assert directory.exists(), f"Directory {directory} does not exist"
        self.directory = directory
        self.thread = None
        self.httpd = None
        self.public_url = None
        self.unc_share = None
        self.prefix = 'images'
        self.tunnel = None
        app.config['image_dir'] = str(directory)  # directory for Bottle
        # app.routes[0].callback.__globals__['image_dir'] = str(directory)  # directory for Bottle

    @property
    def running(self):
        return self.thread is not None and self.thread.is_alive()

    def start_server(self):
        def bottle_run():
            run(app, host='localhost', port=self.port)
        
        self.thread = threading.Thread(target=bottle_run)
        self.thread.start()
        self.tunnel = ngrok.connect(self.port)  # type: ignore
        self.public_url = self.tunnel.public_url
        if self.public_url is not None:
            self.unc_share = Path(self.public_url.replace('https:', ''))/self.prefix
        cprint(f"ngrok tunnel: {self.tunnel}")
        cprint(f"Public URL: {self.public_url}")

    def start(self):
        if self.thread is None or not self.thread.is_alive():
            self.start_server()
        else:
            cprint("Server is already running")

    def stop(self):
        if self.tunnel and self.tunnel.public_url:
            ngrok.disconnect(self.tunnel.public_url)
            cprint("Ngrok tunnel disconnected")
            ngrok.kill()
        
        if self.thread:
            self.make_request('/shutdown')
            self.thread.join(timeout=10)
            if self.thread.is_alive():
                print("Thread did not terminate, proceeding with forceful shutdown.")
            else:
                print("Server thread stopped successfully.")
        self.thread = self.tunnel = self.public_url = self.unc_share = None
        cprint("Server stopped")

    def make_request(self, path="/"):
        """Makes a request to the ngrok URL with headers to bypass the ngrok warning."""
        if self.public_url:
            url = f"{self.public_url}{path}"
            headers = {
                "ngrok-skip-browser-warning": "true",
                "User-Agent": "MyCustomUserAgent/1.0"
            }
            response = requests.get(url, headers=headers)
            return response.text
        else:
            return "Server not started or public URL not available."

    def __enter__(self):
        self.start()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.stop()


In [ ]:
serve_dir = cache_dir
test_eq(serve_dir.exists(), True)
serve_dir

Path('../experiment/cache')

In [ ]:
server = setup_ngrok(WebServerBottle, serve_dir)


Enter your ngrok authtoken, which can be copied from 
https://dashboard.ngrok.com/get-started/your-authtoken

Bottle v0.13-dev server starting up (using WSGIRefServer())...
Listening on http://localhost:55470/
Hit Ctrl-C to quit.



ngrok tunnel: NgrokTunnel: "https://0836-83-33-227-209.ngrok-free.app" -> "http://localhost:55470"

Public URL: https://0836-83-33-227-209.ngrok-free.app

127.0.0.1 - - [21/May/2024 13:24:01] "GET /images/pcleaner.png HTTP/1.1" 200 17709
127.0.0.1 - - [21/May/2024 13:24:45] "GET /images/Strange_Tales_172005/.crop/Strange_Tales_172005_0_Default.png HTTP/1.1" 200 137784
127.0.0.1 - - [21/May/2024 13:24:48] "GET /images/images/Strange_Tales_172005/.crop/Strange_Tales_172005_1_Default.png HTTP/1.1" 404 817
127.0.0.1 - - [21/May/2024 13:25:01] "GET /images/Strange_Tales_172005/.crop/Strange_Tales_172005_1_Default.png HTTP/1.1" 200 107550


In [ ]:
assert server is not None and server.public_url is not None
public_url: str = f"{server.public_url}/{server.prefix}"


In [ ]:
img_path = 'Strange_Tales_172005/.crop/Strange_Tales_172005_0_Default.png'


In [ ]:
f'<img src="{public_url}/{img_path}"/>'

'<img src="https://0836-83-33-227-209.ngrok-free.app/images/Strange_Tales_172005/.crop/Strange_Tales_172005_0_Default.png"/>'

In [ ]:
display(HTML(f'<img src="{public_url}/{img_path}"/>'))


In [ ]:
display(HTML(f'<img src="{public_url}/Strange_Tales_172005/.crop/Strange_Tales_172005_1_Default.png"/>'))


In [ ]:
if server is not None: 
    server.stop()


t=2024-05-21T13:25:04+0200 lvl=warn msg="Stopping forwarder" name=http-55470-05e35e60-2237-402e-b7ab-8507dd7f47dc acceptErr="failed to accept connection: Listener closed"


Ngrok tunnel disconnected

Thread did not terminate, proceeding with forceful shutdown.


Server stopped

In [ ]:
test_eq(ngrok.get_tunnels(), [])

In [ ]:
PORT = server.port
PORT


58805

In [ ]:
_PID = !lsof -ti :$PORT  # Find the process using PORT  # type: ignore
if len(_PID) > 0: _PID = _PID[0]
_PID


[]

In [ ]:
# !kill -9 $_PID

# Colophon
----


In [ ]:
import fastcore.all as FC
from nbdev.export import nb_export


In [ ]:
if FC.IN_NOTEBOOK:
    nb_export('web_server.ipynb', '..')
